### 다중 선형 회귀 분석
-------------
    다중 선형 회귀 분석은 2개 이상의 독립변수와 하나의 종속변수 간의 관계를 분석하는 방법이다. 중선형 회귀 분석이라고도 부른다. 대부분 독립변수는 2개 이상인 다중 선형 회귀 분석이 사용된다.
   

- 다중 선형 회귀식
      Y = β₀ + β₁X₁ + β₂X₂ + ... + βₙXₙ + ε
  - X:독립변수
  - Y:종속변수
  - α:절편
  - β:기울기
  - ϵ: 오차항(모델이 설명하지 못하는 부분)

    종속변수를 가장 잘 설명하는 회귀 계수(기울기,절편 등)들을 추정한다.


In [1]:
import pandas as pd
data = {
    '매출액': [300, 320, 250, 360, 315, 328, 310, 335, 326, 280,
            290, 300, 315, 328, 310, 335, 300, 400, 500, 600],
    '광고비': [70, 75, 30, 80, 72, 77, 70, 82, 70, 80,
            68, 90, 72, 77, 70, 82, 40, 20, 75, 80],
    '직원수': [15, 16, 14, 20, 19, 17, 16, 19, 15, 20,
            14, 5, 16, 17, 16, 14, 30, 40, 10, 50]
    }
df = pd.DataFrame(data)
print(df.head(3))

   매출액  광고비  직원수
0  300   70   15
1  320   75   16
2  250   30   14


#### 1. 주어진 데이터로 최소제곱법을 이용한 다중 선형 회귀 모델을 구축하고 통계적 요약을 출력하시오.


In [5]:
from statsmodels.formula.api import ols
model = ols('매출액~광고비+직원수',data=df).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                    매출액   R-squared:                       0.512
Model:                            OLS   Adj. R-squared:                  0.454
Method:                 Least Squares   F-statistic:                     8.907
Date:                Tue, 12 Nov 2024   Prob (F-statistic):            0.00226
Time:                        04:23:35   Log-Likelihood:                -108.22
No. Observations:                  20   AIC:                             222.4
Df Residuals:                      17   BIC:                             225.4
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    101.0239     71.716      1.409      0.1

In [7]:
 print("광고비-매출액 상관계수 :", df['광고비'].corr(df['매출액']))

광고비-매출액 상관계수 : 0.13316981737040343


In [10]:
from scipy import stats
print('\n광고비와 매출액의 t검정의 p-value:')
print(stats.pearsonr(df['광고비'],df['매출액']))


광고비와 매출액의 t검정의 p-value:
PearsonRResult(statistic=0.13316981737040345, pvalue=0.5756778801904272)


In [11]:
print("결정계수 : ",model.rsquared)

결정계수 :  0.5116964327009041


In [12]:
print("회귀 계수 : \n",round(model.params))

회귀 계수 : 
 Intercept    101.0
광고비            2.0
직원수            6.0
dtype: float64


In [14]:
print("pvalue:",model.pvalues['광고비'])

pvalue: 0.03764350647695994


Q. 광고비 50, 직원수 20인 데이터가 존재할때, 구축한 회귀모델에서의 예측 매출액을 구하시오

In [20]:
new_data = pd.DataFrame({"광고비":[50],"직원수":[20]})
result = model.predict(new_data)
print("새로운 데이터의 예측 매출액 : ",result)

새로운 데이터의 예측 매출액 :  0    310.57033
dtype: float64


In [22]:
df['잔차'] = df['매출액'] - model.predict(df)
print('잔차의 제곱합: ',sum(df['잔차']**2))

잔차의 제곱합:  58686.178271561075


In [26]:
print('잔차의 제곱합: ',sum(model.resid**2))

잔차의 제곱합:  58686.178271561075


In [27]:
MSE = (df['잔차']**2).mean()
print("평균제곱오차 (MSE):",MSE)

평균제곱오차 (MSE): 2934.3089135780533


##### 각 변수별 95% 신뢰구간 , 다른 % 구간을 구하기 위해서는
```Python
model.conf_int(alpha=0.05)
```

In [29]:
print('95% 신뢰구간:\n',model.conf_int(alpha=0.05))

95% 신뢰구간:
                    0           1
Intercept -50.283684  252.331429
광고비         0.116785    3.522069
직원수         2.912406    8.945105


Q. 광고비 45, 직원수 22인 데이터가 존재할때, 구축한 회귀모델에서의 예측 매출액을 구하시오

In [37]:
new_data = pd.DataFrame({'광고비':[45],'직원수':[22]})
pred = model.get_prediction(new_data)
result = pred.summary_frame(alpha=0.05)
print("95% 신뢰구간에서의 예상 매출액 : obs_ci-> \n ",result)

95% 신뢰구간에서의 예상 매출액 : obs_ci-> 
           mean    mean_se  mean_ci_lower  mean_ci_upper  obs_ci_lower  \
0  313.330707  22.502058     265.855514       360.8059     180.58875   

   obs_ci_upper  
0    446.072663  


##### 회귀 분석에서 독립변수로 수치형변수와 범주형 변수를 함께 사용할 경우에는 범주형 변수는 통상적으로 원-핫 인코딩을 사용해 수치화한다.


In [38]:
from google.colab import files
uploaded = files.upload()

Saving study.csv to study.csv


In [40]:
import pandas as pd
import statsmodels.api as sm
df = pd.read_csv('study.csv')
df.head()

,study_hours,material_type,score
0,71,강의,95
1,34,독학,63
2,91,도서,95
3,80,독학,80
4,40,강의,79


In [41]:
from statsmodels.formula.api import ols
model = ols('score ~ study_hours + material_type',data=df).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  score   R-squared:                       0.969
Model:                            OLS   Adj. R-squared:                  0.968
Method:                 Least Squares   F-statistic:                     991.9
Date:                Tue, 12 Nov 2024   Prob (F-statistic):           4.42e-72
Time:                        04:42:18   Log-Likelihood:                -238.89
No. Observations:                 100   AIC:                             485.8
Df Residuals:                      96   BIC:                             496.2
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              59.2111    